In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sejong-ai-2021-19011824/Submission.csv
/kaggle/input/sejong-ai-2021-19011824/train.csv
/kaggle/input/sejong-ai-2021-19011824/test.csv


In [2]:
train=pd.read_csv("/kaggle/input/sejong-ai-2021-19011824/train.csv")
test=pd.read_csv("/kaggle/input/sejong-ai-2021-19011824/test.csv")
submit=pd.read_csv("/kaggle/input/sejong-ai-2021-19011824/Submission.csv")

In [3]:
test=test.drop(["Unnamed: 0"],axis=1)
x=train.drop(["Unnamed: 0","Tsu","Eq"],axis=1)

In [4]:
label=[]
tsu=train["Tsu"]
Eq=train["Eq"]

for i in range(len(train)):
  if tsu[i]==1 and Eq[i]==1:
    label.append(3)
  elif tsu[i]==1 and Eq[i]==0:
    label.append(2)
  elif tsu[i]==0 and Eq[i]==1:
    label.append(1)
  else:
    label.append(0)

label=pd.DataFrame(label)
y=label[0]

In [5]:
con_list=["Name","Location","Country","Type","Agent"]

from sklearn import preprocessing

lable_encoder=pd.concat([x,test])

for i in con_list:
  le = preprocessing.LabelEncoder()
  lable_encoder[i]=lable_encoder[i].astype("str")
  x[i]=x[i].astype("str")
  test[i]=test[i].astype("str")
  le.fit(lable_encoder[i])
  x[i]=le.transform(x[i])
  test[i]=le.transform(test[i])
  

In [6]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
x['Mo']=imp_mean.fit_transform(x['Mo'].to_numpy().reshape(-1, 1))
test['Mo']=imp_mean.transform(test['Mo'].to_numpy().reshape(-1, 1))

x['Dy']=imp_mean.fit_transform(x['Dy'].to_numpy().reshape(-1, 1))
test['Dy']=imp_mean.transform(test['Dy'].to_numpy().reshape(-1, 1))


In [7]:
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler

transformer =StandardScaler().fit(x)
x=transformer.transform(x)
test=transformer.transform(test)


In [8]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='constant',fill_value=0)
imp_mean.fit(x)
x=imp_mean.transform(x)
test=imp_mean.transform(test)

In [9]:
import torch
seed=0
torch.manual_seed(seed)

device='cuda' if torch.cuda.is_available() else 'CPU'

if device=='cuda':
  torch.cuda.manual_seed_all(seed)

x=np.array(x)
y=np.array(y)
test=np.array(test)

x=torch.FloatTensor(x)
y=torch.LongTensor(y)
test=torch.FloatTensor(test)

In [10]:
linear1=torch.nn.Linear(32,128)
linear2=torch.nn.Linear(128,128)
linear3=torch.nn.Linear(128,64)
linear4=torch.nn.Linear(64,4)

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)

relu=torch.nn.ReLU()

dropout=torch.nn.Dropout(p=0.1)

In [11]:
model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4)
optimizer = torch.optim.Adam(params= model.parameters(), lr=0.001)
loss=torch.nn.CrossEntropyLoss()

In [12]:
model.train()
for stop in range(1000):
  optimizer.zero_grad()
  h=model(x)
  cost=loss(h,y)
  cost.backward()
  optimizer.step()
  if stop%100==0:
    print(stop,cost.item())

0 1.4662742614746094
100 0.2680996358394623
200 0.05807718262076378
300 0.026537302881479263
400 0.013720749877393246
500 0.007555725984275341
600 0.006013635080307722
700 0.006526468321681023
800 0.0013413975248113275
900 0.002128224354237318


In [13]:
pred=list()
with torch.no_grad():
  
  model.eval()
  test_y=model(test)
  a,b=torch.max(test_y,dim=1)
  pred.extend(b.tolist())

In [14]:
submit["label"]=pred
submit.to_csv("submit.csv",index=False)